In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
import ipywidgets as widgets
from rt2_assignment1.srv import Command
from IPython.display import display
from tf import transformations
import math

rospy.init_node('robot_control_node')

# start/stop the robot
## user interface
- starting/ stopping the robot "random position" behaviour by using two buttons

In [2]:
ui_client = rospy.ServiceProxy('/user_interface', Command)


start_button = Button(description='start', layout=Layout(width='auto', align="center", grid_area='b1'), style=ButtonStyle(button_color='lightblue')) 


stop_button = Button(description='stop', layout=Layout(width='auto', grid_area='b2'), style=ButtonStyle(button_color='moccasin')) 

output = widgets.Output() 
moving_random_pos=0

def click_start(change):
    global moving_random_pos
    with output:
        if moving_random_pos==0:
            ui_client("start")
            print('the robot starts to move to the random position')
            moving_random_pos=1
        else:
            print('the robot is going to the previous random position')
        
def click_stop(change):
    global moving_random_pos
    with output:
        print('stop')
        if moving_random_pos==1:
            ui_client("stop")
            print('click the stop button')
        moving_random_pos=0

start_button.on_click(click_start)
stop_button.on_click(click_stop)

HBox([start_button,stop_button])

# set the cmd_vel
- setting the linear and angular velocity by using two Sliders 

In [3]:
msg = Twist()

a = widgets.FloatSlider(
    min=-1.0,
    max=1.0,
    step=0.1,
    description='linear velocity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',)

display(a)

b = widgets.FloatSlider(
    min=-1.0,
    max=1.0,
    step=0.1,
    description='angular velocity:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',)

display(b)

pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

def linear_vel_change(change):
    global msg,moving_random_pos
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x = change['new']
    pub.publish(msg)
    
def angular_vel_change(change):
    global msg,moving_random_pos
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.angular.z = change['new']
    pub.publish(msg)

a.observe(linear_vel_change, names='value')
b.observe(angular_vel_change, names='value')

FloatSlider(value=0.0, continuous_update=False, description='linear velocity:', max=1.0, min=-1.0, readout_for…

FloatSlider(value=0.0, continuous_update=False, description='angular velocity:', max=1.0, min=-1.0, readout_fo…

the robot is going to the previous random position and the goal is canceled


#  robot controller
- directly controlling the robot movements by using 5 Buttons, (forward, turn right, backward, turn left, stop), possibly placed in an intuitive fashion 

In [4]:
left_button = Button(description='←', layout=Layout(width='auto', grid_area='left'), style=ButtonStyle(button_color='salmon')) 
up_button = Button(description='↑',layout=Layout(width='auto', grid_area='up'), style=ButtonStyle(button_color='salmon')) 
right_button = Button(description='→', layout=Layout(width='auto', grid_area='right'), style=ButtonStyle(button_color='salmon')) 
down_button = Button(description='↓', layout=Layout(width='auto', grid_area='down'), style=ButtonStyle(button_color='salmon')) 

def click_up(change):
    global moving_random_pos,msg
    print("Go straight")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x=0.5
    msg.linear.y=0.0
    msg.angular.z=0.0
    pub.publish(msg)
        
def click_down(change):
    global moving_random_pos,msg
    print("Go back")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x=-0.5
    msg.linear.y=0.0
    msg.angular.z=0.0
    pub.publish(msg)

def click_right(change):
    global moving_random_pos,msg
    print("Go right")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
        moving_random_pos=0
    msg.linear.x=0.1
    msg.linear.y=0.0
    msg.angular.z=0.5
    pub.publish(msg)

def click_left(change):
    global moving_random_pos,msg
    print("Go left")
    if moving_random_pos==1:
        print('the robot is going to the previous random position and the goal is canceled')
        ui_client("stop")
    moving_random_pos=0
    msg.linear.x=0.1
    msg.linear.y=0.0
    msg.angular.z=-0.5
    pub.publish(msg)
            
up_button.on_click(click_up)
down_button.on_click(click_down)
right_button.on_click(click_right)
left_button.on_click(click_left)

GridBox(children=[left_button,down_button,right_button, up_button], 
        layout=Layout( 
            width='20%', 
            grid_template_rows='auto auto', 
            grid_template_columns='33% 33% 33%', 
            grid_template_areas=''' 
            "left up right" 
            ". down . " 
            ''') 
       ) 

GridBox(children=(Button(description='←', layout=Layout(grid_area='left', width='auto'), style=ButtonStyle(but…

Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right
Go right


# visualize the trajectory of the robot
- a line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity) 
https://python.atelierkobato.com/annotate/
https://www.it-swarm-ja.com/ja/python/python-matplotlib%E3%82%A2%E3%83%8B%E3%83%A1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E3%81%A7%E5%81%9C%E6%AD%A2%E9%96%8B%E5%A7%8B%E4%B8%80%E6%99%82%E5%81%9C%E6%AD%A2/1073484243/

In [5]:
# First set up the figure and the axis 
fig, ax = plt.subplots()

ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))


line, = ax.plot([], [], 'ro')
x_pos=0
y_pos=0
x_data=[]
y_data=[]
yaw=0

def init():
    line.set_data([], [])
    return (line,)


def odom_callback(msg):
    global x_pos,y_pos
    x_pos=msg.pose.pose.position.x
    print(x_pos)
    y_pos=msg.pose.pose.position.y
    y_data.append(y_pos)
    x_data.append(x_pos)
    
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]
    

def animate(i):
    global x_pos,y_pos,yaw
    line.set_data(x_data, y_data)
    arrow = plt.Arrow(x=x_pos,y=y_pos,dx=np.cos(yaw),dy=np.sin(yaw),width=0.01,head_width=0.05,head_length=0.2,length_includes_head=True,color='k')
    ax.add_patch(arrow)
    return (line, arrow)



jr.subscribe('/odom', Odometry, odom_callback)

# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …